# **0.Librairies**

In [97]:
# Data manipulation
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, f1_score,
                             make_scorer, roc_auc_score, roc_curve)
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     cross_val_score, cross_validate,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

# Scipy
from scipy.stats import ks_2samp

# Imbalanced-learn
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# XGBoost and LightGBM
import xgboost as xgb
import lightgbm as lgb

# PyTorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# UCI ML Repo
from ucimlrepo import fetch_ucirepo

# Custom scripts
from scripts.data_preprocessing import fetch_data
from scripts.data_visualisation import data_visualisation

# **I. Problem Setting**

### **1. Problem Definition**

We will use the [Online Shoppers Purchasing Intention](https://archive.ics.uci.edu/dataset/468/online+shoppers+purchasing+intention+dataset) dataset to predict whether a user's session will result in a purchase or not. This is a **binary classification problem**.

- **Goal:** Build a predictive model that can classify user sessions into two categories:
    - **Revenue = True**: the session results in a purchase.
    - **Revenue = False**: the session does not result in a purchase.

### **2. Mathematical Formulation**

Let:
- A dataset $X=\{x_1,x_2,...,x_n\}$ where each $x_i$ is a feature vector representing a session.
- A target variable $Y=\{y_1, y_2, ..., y_n\}$ where $y_i \in \{0, 1\}$ (0 for "no purchase", 1 for "purchase").

The goal is to find a function $f : X \rightarrow Y$ such that for a new example $x$, $f(x)$ correctly predicts $y$.

# **II. Data Analysis**

The dataset consists of feature vectors belonging to 12,330 sessions. 

The dataset was formed so that each session would belong to a different user in a 1-year period to avoid any tendency to a specific campaign, special day, user profile, or period.

It cointains no missing value.

In [149]:
# Import the dataset using fetch_ucirepo function from the UCIML repository
online_shoppers_purchasing_intention_dataset = fetch_ucirepo(id=468) 

# Retrieve the variables/metadata associated with the dataset
variables = online_shoppers_purchasing_intention_dataset.variables 

# Extract the feature matrix (X) and target vector (y) from the dataset
X = online_shoppers_purchasing_intention_dataset.data.features 
y = online_shoppers_purchasing_intention_dataset.data.targets

In [147]:
# Save Data
# variables.to_csv('../data/online_shoppers_purchasing_intention_dataset.csv', index=False)
# X.to_csv('../data/features.csv', index=False)
# y.to_csv('../data/targets.csv', index=False)

# Load Data
variables = pd.read_csv('../data/online_shoppers_purchasing_intention_dataset.csv')
X = pd.read_csv('../data/features.csv')
y = pd.read_csv('../data/targets.csv')

## **1. Exploratory Data Analysis (EDA)**

This section is dedicated to the **general understanding of the variables** and will consist of the study of:
- **Numerical variables:** Analysis of distributions (mean, median, standard deviation).
- **Categorical variables:** Counting occurrences, encoding.

### **a. Variables Dataset**

In [ ]:
variables.info()

In [101]:
# We remove empty columns
variables = variables[['name', 'role', 'type', 'missing_values']]

In [ ]:
variables

### **b. Target: Revenue (y)**

In [ ]:
y.info()

In [ ]:
y.describe()

In [ ]:
print(y['Revenue'].sum()/len(y))

# On affiche la distribution de la variable cible
sns.countplot(x='Revenue', data=y)
plt.show()

We observe that only **14,5%** of users bought.

### **c. Features (X)**

#### General Data

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X.describe()

There is indeed no missing values.

## **2. Initial Visualisations**

### **a. Visualisation of numerical features**

In [ ]:
# Select the numeric features from the dataset
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()

# Determine the number of rows and columns for subplots
n = len(numeric_features)
cols = 4  # Number of columns for the grid layout
rows = n // cols + (n % cols > 0)  # Number of rows, accounting for remaining features

# Create the figure and subplots
fig, axes = plt.subplots(rows, cols, figsize=(16, 12))  # Define the figure size
axes = axes.flatten()  # Flatten the axes array to easily iterate through

# Plot each numeric feature's distribution
for i, feature in enumerate(numeric_features):
    sns.histplot(X[feature], kde=True, ax=axes[i])  # Plot histogram and KDE for each feature
    axes[i].set_title(f'Distribution of {feature}')  # Set title for each subplot

# Remove unused subplots if there are any extra axes
for i in range(n, len(axes)):
    fig.delaxes(axes[i])

# Adjust the layout for better readability
plt.tight_layout()
plt.show()  # Display the figure

- Most of the distributions show a high concentration of data **close to zero**, particularly for interactions with specific types of pages (administrative, informational, and product-related pages). We can assume that only a small fraction of users engage with these types of pages.

- Analyzing the exit and bounce rates could help **identify engagement issues**, especially if certain types of pages (such as product or informational pages) have higher exit rates.

In [ ]:
# Select the categorical features (object or boolean type) from the dataset
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()

# Determine the number of categorical features
n = len(categorical_features)
cols = 3  # Number of columns for the grid layout

# Create the figure and subplots
fig, axes = plt.subplots(1, cols, figsize=(16, 4))  # Define the figure size for a single row
axes = axes.flatten()  # Flatten the axes array to iterate through

# Plot the distribution of each categorical feature
for i, feature in enumerate(categorical_features):
    sns.histplot(X[feature], kde=True, ax=axes[i])  # Plot histogram for each feature
    axes[i].set_title(f'Distribution of {feature}')  # Set the title for each subplot

# Remove unused subplots if there are any extra axes
for i in range(n, len(axes)):
    fig.delaxes(axes[i])

# Adjust the layout for better readability
plt.tight_layout()
plt.show()  # Display the figure

Several observations can be made:
- There is a **seasonality pattern**, with peaks in the spring and fall.

- A stronger presence of **returning visitors** indicates that the site has a good retention rate.

- Purchases appear to be **evenly distributed** throughout the week, with a consistent proportion of about 5/7 on weekdays and 2/7 on weekends. This might suggest that the site is used equally for both professional and recreational purposes.

There are no missing values in either the **features** or the **target** dataset, so we can directly proceed with working on the data.

#### **b. Correlation Matrix :**
To identify linked data.

In [ ]:
# Create a figure with a specified size
plt.figure(figsize=(12, 10))

# Compute the correlation matrix for the numeric features
correlation = X[numeric_features].corr()

# Create a heatmap to visualize the correlation matrix
sns.heatmap(correlation, annot=True, cmap='coolwarm')  # 'coolwarm' colormap for better visualization

# Add a title to the plot
plt.title('Correlation Matrix')

# Display the plot
plt.show()

**Strong Correlations:**

- **ProductRelated** and **ProductRelated_Duration** (0.86): It makes sense that the number of product pages visited is strongly correlated with the time spent on those pages. The more product pages users visit, the more time they spend on those pages.

- **BounceRates** and **ExitRates** (0.91): These two variables are also highly correlated. This indicates that sessions with a high bounce rate tend to end quickly (high probability of leaving the site after viewing few pages).

**Moderate Correlations:**

- **Informational** and **Informational_Duration** (0.62): Similarly, there is a fairly high correlation between the number of informational pages visited and the time spent on those pages, which is also intuitive.

- **Administrative** and **Administrative_Duration** (0.6): The relationship between the number of administrative pages visited and the time spent on those pages is moderate. The more users interact with administrative pages, the more time they spend on them.


We will use `sns.violinplot` to observe the distributions of **feature** against **target**.

In [ ]:
# Function to filter the data based on the specified percentiles for a given feature
def filter_percentiles(df, feature, lower_percentile=0.10, upper_percentile=0.90):
    lower_bound = df[feature].quantile(lower_percentile)
    upper_bound = df[feature].quantile(upper_percentile)
    # Return the filtered dataframe based on the lower and upper percentiles
    return df[(df[feature] >= lower_bound) & (df[feature] <= upper_bound)]

data = pd.concat([X, y], axis=1)
plt.figure(figsize=(16, 12))

for i, feature in enumerate(numeric_features, 1):
    plt.subplot(4, 4, i) 
    # Filter the data based on the 10th and 90th percentiles of the feature
    data_filtered = filter_percentiles(data, feature)
    sns.violinplot(x='Revenue', y=feature, data=data_filtered, split=True) 
    plt.title(f'{feature} vs Revenue (80% values)')  

plt.tight_layout()
plt.show()

- **Product Pages**: Variables related to product pages (`ProductRelated` and `ProductRelated_Duration`) are the best indicators for distinguishing users who generate revenue from those who don't. The more a user interacts with product pages, the more likely they are to generate revenue.

- **Bounce Rates** and **Exit Rates**: Users who generate revenue tend to have lower bounce and exit rates, indicating deeper engagement with the site.

- **PageValues**: This is one of the most discriminative variables, with users generating revenue having much higher page values.

- Other factors do not seem to have a significant impact on revenue generation, at least in this visualization.

# **III. Modelisation**

## **1. Data Pre-processing**

### **a. Categorial Variables Encoding**

Converting `bool` into `int` :

In [114]:
data['Weekend'] = data['Weekend'].astype(int)
data['Revenue'] = data['Revenue'].astype(int)

One-Hot Encoding for multi-categorial data :

In [115]:
data = pd.get_dummies(data, columns=['Month', 'VisitorType'])

In [ ]:
data.head()

### **b. Numeric Variables Standardisation**
Unifying the scale of the Data :

In [117]:
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

### **c. Class Unbalance**
SMOTE to over-sample the minoritary class :

In [118]:
X = data.drop('Revenue', axis=1)
y = data['Revenue']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

### **d. Division into training and testing sets**

In [119]:
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, 
    y_resampled, 
    test_size=0.3, # 30% of the data is used for testing
    random_state=42,
    stratify=y_resampled # Stratification 
    )

## **2. Classification Algorithms**

We will use the following classic methods and then compare their performances:
- Logistic Regression
- Decision Tree
- Random Forest
- XGBoost
- Neural Network with PyTorch

### **a. Models Implementation**

#### Logistic Regression

In [120]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

#### Decision Tree

In [121]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

#### Random Forest

In [122]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

#### XGBoost

In [123]:
xgb_model = xgb.XGBClassifier(eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

#### Neural Network with PyTorch

In [124]:
# Convert the variables to a tensor
X_train_tensor = torch.tensor(X_train.values.astype(float), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values.astype(float), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create a TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [125]:
class NeuralNet(nn.Module):
    # Define the layers and activation functions in the constructor
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, 64) # Input layer with 64 neurons
        self.layer2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 2) # Output layer with 2 classes
        self.relu = nn.ReLU()
    
    # Define the forward pass
    def forward(self, x):
        out = self.relu(self.layer1(x)) # ReLU activation
        out = self.relu(self.layer2(out)) # ReLU activation
        out = self.output(out)
        return out

# Instantiate the model, define the loss function and optimizer
model = NeuralNet(input_size=X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train the model
num_epochs = 20 # Number of epochs

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        outputs = model(inputs) # Forward pass
        loss = criterion(outputs, labels) # Calculate the loss
        
        optimizer.zero_grad() # Zero the gradients
        loss.backward() # Backpropagation
        optimizer.step() # Update the weights
    
    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
# Evaluate the model
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    all_preds = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0) 
        correct += (predicted == labels).sum().item() 
        all_preds.extend(predicted.numpy()) 
    
    print(f'Accuracy du réseau sur les données de test : {100 * correct / total}%')

### **b. Model Evaluation**

Precision alone loses reliability in a scenario where the classes are imbalanced. Therefore, we will use 3 evaluation methods that are particularly relevant in this case, and are imported from `sklearn.metrics`:

- `classification_report`:
  - the **precision** of the model (proportion of positive predictions that are correct).
  - the **recall** (proportion of actual positives that are correctly predicted).
  - the **f1-score** (harmonic mean of precision and recall).

- `confusion_matrix`: displays the **True Positives**, **True Negatives**, **False Positives**, and **False Negatives** within a 2x2 matrix.

- `roc_auc_score`: **ROC** (Receiver Operating Characteristic) is the plot of the **True Positive Rate** against the **False Positive Rate** for each possible classification threshold between 0 and 1 (in practice, selected intervals). **AUC** is the area under the ROC curve; the closer it is to 1, the better the model performs. If it is close to 0.5, it is equivalent to random classification.


In [172]:
# Dictionary to store predictions from different models
predictions = {
    'Logistic Regression': y_pred_lr,  # Predictions from Logistic Regression
    'Decision Tree': y_pred_dt,        # Predictions from Decision Tree
    'Random Forest': y_pred_rf,        # Predictions from Random Forest
    'XGBoost': y_pred_xgb,             # Predictions from XGBoost
    'NN (PyTorch)': all_preds  # Predictions from the PyTorch Neural Network
}


#### **Classification Report**

In [ ]:
# List to store results for each model
results = []

# Loop through each model and its predictions
for model_name, y_pred in predictions.items():
    # Get the classification report as a dictionary
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Extract precision, recall, and f1-score for both classes (0 and 1)
    precision_0 = report['0']['precision']
    precision_1 = report['1']['precision']
    recall_0 = report['0']['recall']
    recall_1 = report['1']['recall']
    f1_0 = report['0']['f1-score']
    f1_1 = report['1']['f1-score']

    # Append the results for each model to the results list
    results.append({
        'Model': model_name,
        'Precision False': precision_0,
        'Precision True': precision_1,
        'Recall False': recall_0,
        'Recall True': recall_1,
        'F1-Score False': f1_0,
        'F1-Score True': f1_1
    })

# Create a DataFrame to display the results
df_results = pd.DataFrame(results)
df_results  # Display the results DataFrame

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

metrics = ['Precision False', 'Recall False', 'F1-Score False', 'Precision True', 'Recall True', 'F1-Score True']
titles = ['Precision False', 'Recall False', 'F1-Score False', 'Precision True', 'Recall True', 'F1-Score True']

for i, metric in enumerate(metrics):
    row, col = divmod(i, 3)  
    sns.barplot(x='Model', y=metric, data=df_results, ax=axes[row, col], palette='viridis', hue='Model')
    axes[row, col].tick_params(axis='x', rotation=30)
    axes[row, col].set_title(titles[i])
    axes[row, col].set_ylabel('Score')
    axes[row, col].set_xlabel('Model')
    axes[row, col].set_ylim((0.8, 1))

plt.tight_layout()
plt.show()

The **Random Forest** seems to achieve better results.

#### **Confusion Matrix**

In [ ]:
# Create a figure with a 2x3 grid of subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Loop through each model and its predictions to plot the confusion matrices
for i, (model_name, y_pred) in enumerate(predictions.items()):
    ax = axes[i // 3, i % 3]  # Determine the correct subplot location
    cm = confusion_matrix(y_test, y_pred)  # Compute the confusion matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='plasma', ax=ax)  # Plot the confusion matrix heatmap
    ax.set_title(f'Confusion Matrix - {model_name}')  # Set title
    ax.set_xlabel('Predictions')  # Set x-axis label
    ax.set_ylabel('True Labels')  # Set y-axis label

# Remove the unused last subplot (bottom right)
axes[1][2].axis('off')

# Adjust layout to prevent overlap and show the plot
plt.tight_layout()
plt.show()

The **Random Forest** and **XGBoost** slightly better in limiting False Positives and False Negatives.

#### **ROC-AUC Score**

To plot the **ROC curve** at different classification thresholds, we calculate the probabilities of belonging to each class for each observation. In practice, we are only interested in the probability $P(class=1)$. To obtain these probabilities, we can use the `predict_proba` method from `sklearn`. However, our **Neural Network (NN)** does not have this method, so we will compute the **logits** and then convert them to probabilities using the **sigmoid** function.

In [ ]:
# Convert the test set to a PyTorch tensor
X_test_tensor = torch.FloatTensor(X_test.values.astype(float))

# Perform inference with the neural network and calculate probabilities using the sigmoid function
with torch.no_grad():
    logits = model(X_test_tensor)  # Get the raw logits from the model
    probs_pytorch = F.sigmoid(logits).numpy()[:, 1]  # Apply sigmoid and convert to probabilities for class 1

# Dictionary to store the predicted probabilities for each model
probabilities = {
    'Logistic Regression': lr.predict_proba(X_test)[:, 1],  # Probabilities for Logistic Regression
    'Decision Tree': dt.predict_proba(X_test)[:, 1],        # Probabilities for Decision Tree
    'Random Forest': rf.predict_proba(X_test)[:, 1],        # Probabilities for Random Forest
    'XGBoost': xgb_model.predict_proba(X_test)[:, 1],       # Probabilities for XGBoost
    'Neural Network (PyTorch)': probs_pytorch               # Probabilities from the PyTorch Neural Network
}

# Create a figure for plotting the ROC curves
plt.figure(figsize=(10, 8))

# Loop through each model and its probabilities to plot the ROC curve
for model_name, probs in probabilities.items():
    fpr, tpr, thresholds = roc_curve(y_test, probs)  # Calculate FPR and TPR
    roc_auc = roc_auc_score(y_test, probs)  # Calculate the AUC score
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')  # Plot the ROC curve

# Plot the random guessing line
plt.plot([0, 1], [0, 1], 'k--')

# Set labels and title
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curves for Multiple Models')

# Add legend and grid
plt.legend(loc='lower right')
plt.grid(True)

# Show the plot
plt.show()

This graph shows that the **Random Forest**, **XGBoost**, and **Neural Networks** models outperform the others in terms of their ability to distinguish between classes, with AUC values close to 1.

Moving forward, we will focus on the **Random Forest** model, as it provides the best overall results.

## **3. Hyperparameters Optimisation**

### **a. Grid Search for Random Forest**

`Grid Search` is a technique used to find the optimal hyperparameters of a machine learning model by testing all possible combinations of the specified hyperparameters and determining which combination provides the best performance based on a specific metric (such as precision, recall, or F1-score).

In [ ]:
# Parameters to test in Grid Search
param_grid = {
    'n_estimators': [100, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20],  # Maximum depth of the tree
    'class_weight': ['balanced', 'balanced_subsample']  # Handle class imbalance
}

# Initialize GridSearchCV with Random Forest and the parameter grid
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1', n_jobs=-1)  # F1-score for evaluation
grid_search.fit(X_train, y_train)  # Fit the model using Grid Search

# Get the best Random Forest model after Grid Search
best_rf = grid_search.best_estimator_

# Predict the labels using the best Random Forest model
y_pred_best_rf = best_rf.predict(X_test)

# Print the best hyperparameters found by Grid Search
print("Best parameters:")
pd.DataFrame(grid_search.best_params_, index=["Parameters"])  # Display the best parameters in a DataFrame

In [ ]:
# Dictionary to store predictions for Random Forest models
predictions_rf = {
    'Random Forest': y_pred_rf,  # Original Random Forest
    'Random Forest (Grid Search)': y_pred_best_rf  # Optimized Random Forest (after Grid Search)
}

# List to store results for each Random Forest model
results_rf = []

# Loop through each model's predictions and calculate the classification report metrics
for model_name, y_pred in predictions_rf.items():
    report = classification_report(y_test, y_pred, output_dict=True)  # Get the classification report as a dictionary
    precision_0 = report['0']['precision']  # Precision for class 0
    precision_1 = report['1']['precision']  # Precision for class 1
    recall_0 = report['0']['recall']        # Recall for class 0
    recall_1 = report['1']['recall']        # Recall for class 1
    f1_0 = report['0']['f1-score']          # F1-score for class 0
    f1_1 = report['1']['f1-score']          # F1-score for class 1

    # Append results for each model
    results_rf.append({
        'Model': model_name,
        'Precision False': precision_0,
        'Precision True': precision_1,
        'Recall False': recall_0,
        'Recall True': recall_1,
        'F1-Score False': f1_0,
        'F1-Score True': f1_1
    })

# Convert the results into a DataFrame for better display
df_results_rf = pd.DataFrame(results_rf)
df_results_rf  # Display the DataFrame with the results for both models

The performances are extremely close to our initial model. Only the **F1-Score** shows a noticeable improvement.

### **b. Cross Validation**

We will conduct a **robustness test** using the `Stratified K-Fold Cross-Validation` method.

What it involves:
- **Stratification**: Ensures that each fold has approximately the same proportion of each class as the original dataset.

- **K-Fold**: The dataset is divided into K equally-sized subsets.

- **Cross Validation**: This involves training and testing the model on the different subsets.

The objectives are as follows:
- **More reliable estimation**: Reduces the variance associated with a single train/test split.

- **Efficient use of data**: All observations are used for both training and testing.

- **Detection of overfitting**: Allows us to verify if the model generalizes well.

In [ ]:
# Define the number of folds for Stratified K-Fold Cross-Validation
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# Define the scoring metrics for evaluation
scoring = {
    'precision': 'precision',  # Precision metric
    'recall': 'recall',        # Recall metric
    'f1': 'f1',                # F1-score metric
    'roc_auc': 'roc_auc'       # ROC-AUC metric
}

# Create a pipeline with SMOTE for class imbalance, a scaler (if needed), and the best Random Forest model
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),  # Handle class imbalance with SMOTE
    ('scaler', StandardScaler()),       # Apply standard scaling (if necessary)
    ('classifier', best_rf)             # Use the optimized Random Forest model
])

# Perform cross-validation using the pipeline, Stratified K-Fold, and the defined scoring metrics
cv_results = cross_validate(
    estimator=pipeline,
    X=X,  # Feature set
    y=y,  # Target variable
    cv=skf,  # Stratified K-Fold cross-validation
    scoring=scoring,  # Evaluation metrics
    n_jobs=-1  # Use all available processors
)

# Print the mean and standard deviation for each metric
for metric in scoring.keys():
    scores = cv_results[f'test_{metric}']
    print(f"{metric} : Mean = {scores.mean():.4f}, Standard Deviation = {scores.std():.4f}")

The cross-validation results show that the model is not very stable, and its performance is not very robust, especially when looking at `precision`, `recall`, and `f1` across different subsamples. However, the slightly lower average performance compared to the results on the full dataset can be explained by the fact that the model is being tested on portions of the data that are too small during each iteration.

# **IV. Results Analysis**

## **1. Variables Importance**

In [ ]:
# Get the feature importances from the best Random Forest model
importances = best_rf.feature_importances_

# Sort the feature importances in descending order
indices = np.argsort(importances)[::-1]

# Create a barplot of the top 10 most important features
plt.figure(figsize=(12, 6))
plt.title("Feature Importance")
sns.barplot(x=X_train.columns[indices][:10], y=importances[indices][:10])  # Plot the top 10 features

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Display the plot
plt.show()

`PageValues` is by far the most explanatory variable. This makes sense, as **PageValues** is a value based on the probability that a specific page leads to a conversion, referring to the site's historical data.

We also notice that **categorical variables** are not among the most important features.

## **2. Error Analysis**

We'lle keep only the most important **10 features**.

In [ ]:
top_features = X_train.columns[indices][:10]

# Adding the true and predicted labels to the DataFrame
X_test_df = X_test.copy()[top_features]
X_test_df['True_Label'] = y_test.values
X_test_df['Predicted_Label'] = y_pred_best_rf

# Misclassified instances
misclassified = X_test_df[X_test_df['True_Label'] != X_test_df['Predicted_Label']]
print(f"Nombre d'instances mal classées : {len(misclassified)} sur {len(X_test_df)} total")

# Correctly classified instances
correctly_classified = X_test_df[X_test_df['True_Label'] == X_test_df['Predicted_Label']]

In [ ]:
# Mean and standard deviation for the top features in misclassified instances
misclassified_stats = misclassified[top_features].describe().T[['mean', 'std']]

# Mean and standard deviation for the top features in correctly classified instances
correctly_classified_stats = correctly_classified[top_features].describe().T[['mean', 'std']]

# Stats for comparison
comparison_stats = misclassified_stats.join(correctly_classified_stats, lsuffix='_misclassified', rsuffix='_correctly_classified')
comparison_stats

We will observe the **density** distributions of each feature based on whether they were correctly classified or not.

This may potentially help us identify characteristics in the data that led to misclassification.

In [ ]:
numeric_features = X_test_df.select_dtypes(include=[np.number]).columns.tolist() # Get the numeric features
numeric_features.remove('True_Label') # Remove the 'True_Label' column
numeric_features.remove('Predicted_Label')  # Remove the 'Predicted_Label' column

plt.figure(figsize=(16, 12))
for i, feature in enumerate(numeric_features, 1):
    
    # Compute the min and max values for the x-axis with a margin of 1.96 * std
    # (95% confidence interval for a normal distribution)
    min_val = min(comparison_stats.loc[feature, 'mean_misclassified'] - 1.96 * comparison_stats.loc[feature, 'std_misclassified'],
                  comparison_stats.loc[feature, 'mean_correctly_classified'] - 1.96 * comparison_stats.loc[feature, 'std_correctly_classified'])
    max_val = max(comparison_stats.loc[feature, 'mean_misclassified'] + 1.96 * comparison_stats.loc[feature, 'std_misclassified'],
                  comparison_stats.loc[feature, 'mean_correctly_classified'] + 1.96 * comparison_stats.loc[feature, 'std_correctly_classified'])
    
    # Create a subplot for each feature
    plt.subplot(4, 4, i)
    sns.kdeplot(data=correctly_classified, x=feature, label='Bien classés', fill=True)
    sns.kdeplot(data=misclassified, x=feature, label='Mal classés', fill=True)
    
    # Set the x-axis limits and title for each subplot
    plt.xlim(min_val, max_val)
    plt.title(f'Distribution de {feature}')
    plt.legend()

plt.tight_layout()
plt.show()

If we look only at the four most important variables, we can see that the distributions for `ProductRelated_Duration` and `Administrative` are quite clearly distinct. However, those for `PageValues` and `ExitRates` are not as distinct as we might expect.

We can conduct statistical tests to study these distributions in more detail. If the `p-values` are close to 0, this means that the differences between the distributions are **significant**.

In [ ]:
ks_data = []

for feature in numeric_features:
    # Perform the Kolmogorov-Smirnov test
    stat, p_value = ks_2samp(correctly_classified[feature], misclassified[feature]) 
    ks_data.append([f'{stat:.3f}', f'{p_value:.3f}'])
pd.DataFrame(ks_data, columns=['Statistique KS', 'p-value'], index=numeric_features)

The 8 most important **features** show significantly different distributions.

This highlights some behaviors of our model. For example, for the misclassified cases, the distributions are more spread out with very low and very high values. Additionally, there seems to be a trend where the means of the distributions for the misclassified cases shift more "to the right."

Indeed, as we have already observed in our DataFrame of means and standard deviations, the distributions for the misclassified cases tend to have higher `mean` and `std` values.

We could consider **segmenting** the data based on `PageValues` and training specific models while adding **transformations** to better capture the extremes. The exploration of methods to improve our model's performance will be explored in a second part.

# **V. Conclusion**

We used various machine learning methods to predict users' online purchasing intentions by analyzing the "Online Shoppers Purchasing Intention" dataset. By applying different preprocessing techniques, such as encoding categorical variables, normalizing numerical variables, and handling class imbalance via oversampling with `SMOTE`, we trained several classification models. Among these models, **Random Forest** and **XGBoost** demonstrated the best performance, with high `F1` scores, indicating a good ability to distinguish between sessions that resulted in a purchase and those that did not.

The analysis of variables and errors revealed that some features, such as "`PageValues`" and "`ProductRelated_Duration`", were particularly crucial in making predictions. However, the model showed limitations in correctly classifying certain sessions, especially those exhibiting atypical behaviors or falling at the extremes of the variable distributions.

# **VI. To Go Farther**

Several avenues for improvement can be explored to enhance the robustness and accuracy of the model.

## 1. Data Processing Optimization

- **Advanced Feature Engineering**: Creating new variables by combining existing information could help capture non-linear relationships or complex interactions between variables.

- **Variable Transformation**: Applying logarithmic transformations or robust normalization could improve the handling of extreme values and reduce the impact of outliers on the model.

## 2. Integration of Clustering Methods

- **Session Segmentation**: Using clustering algorithms such as `K-Means` or `DBSCAN`, we could identify homogeneous groups of sessions based on user behavior.

- **Anomaly Detection**: Clustering can also help identify atypical sessions.

## 3. Advanced Class Imbalance Management

- **Ensemble Techniques**: Using methods such as `EasyEnsemble` or `BalancedBaggingClassifier` could improve performance on the minority class by combining the predictions of several models trained on balanced subsamples.

- **Class Weighting**: Adjusting the weights associated with classification errors to penalize false negatives more heavily could improve recall.